# Redes Neuronales - TP2
## Ej 3

Implemente un perceptrón multicapa que aprenda la función lógica XOR de 2 y de 4
entradas (utilizando el algoritmo Backpropagation y actualizando en batch). Muestre
cómo evoluciona el error durante el entrenamiento.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
class perceptron_simple:
    def __init__(self):
        self.pesos = None
        self.lr = None
        self.convergencia = False

    def predict(self, x):
        # x ya debe incluir el bias
        return 1 if np.dot(self.pesos, x) >= 0 else -1 # función signo para predecir

    def calcular_error(self, X, y):
        """calcula el error de clasificación en base a los pesos que tiene guardados, el X e Y. """
        error = 0
        for xi, target in zip(X, y):
            xi_con_bias = np.insert(xi, 0, 1) # agrega el 1 para la entrada de bias
            prediccion = self.predict(xi_con_bias) # hace la predicción con los pesos actuales
            if prediccion != target:
                error += 1
        return error

    def train(self, X, y, epochs=100, lr=0.01):
        """
        X: matriz de entradas (sin bias)
        y: vector de salidas esperadas
        epochs: cantidad de iteraciones sobre el dataset
        lr: learning rate
        """
        self.lr = lr
        self.pesos = np.zeros(X.shape[1] + 1)  # tantos pesos como columnas en X más el bias
        error_por_cambio = [] # vamos guardando el error por cada cambio de peso

        # inicializo los pesos en valores aleatorios entre -1 y 1.
        self.pesos = np.random.rand(X.shape[1] + 1)*2 -1 # tenemos valores de 0 a 2 y le resto 1 para tener negativos y positivos
        
        for _ in range(epochs):
            for xi, target in zip(X, y):
                xi_con_bias = np.insert(xi, 0, 1) # agrega el 1 para la entrada de bias
                prediccion = self.predict(xi_con_bias) # hace la predicción con los pesos actuales
                error = target - prediccion # calcula el error como la diferencia entre el valor esperado y el predicho
                self.pesos += self.lr * error * xi_con_bias
                error_luego_de_cambio = self.calcular_error(X,y)
                error_por_cambio.append(error_luego_de_cambio) # guardo el error cada vez que cambia un peso
                if error_luego_de_cambio < 0.01: # condición de corte: si no hay error, convergió
                    break
            if error_luego_de_cambio < 0.01: # condición de corte: si no hay error, convergió
                self.convergencia = True
                break
        if self.convergencia == True:
            return error_por_cambio
        else:
            return 0
